In [ ]:
!pip install pyngrok neo4j

In [ ]:
!ngrok authtoken 2vZo22yz2eBLFLz4ek7TPcyqtyz_45CTrMySYLdTCVB2KKbfu  // DUMMY TOKEN CREATE YOUR OWN

In [ ]:
# =========================
# 2) Imports & Setup
# =========================
from pyngrok import ngrok
from flask import Flask, request, jsonify
import torch
from neo4j import GraphDatabase
from transformers import AutoModelForCausalLM, AutoTokenizer

# =========================
# 3) Your Quiz Generator
# =========================
class DifficultyQuizGenerator:
    def __init__(self, uri, user, password, hf_token):
        """
        Initialize connection to Neo4j and set up the CodeLlama model.
        """
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.hf_token = hf_token

        # Decide whether to use GPU or CPU
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {self.device}")

        # Load the CodeLlama model
        model_name = "meta-llama/CodeLlama-7b-Instruct-hf"  # Requires valid HF token & access
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, token=self.hf_token)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto",
            token=self.hf_token
        )
        # Note: Do not explicitly call .to(self.device) when device_map="auto" is set

    def close(self):
        """Close the Neo4j connection."""
        self.driver.close()

    def fetch_topic_knowledge(self, topic):
        """
        Tries to retrieve text containing the topic (case-insensitive).
        If no direct match is found, attempt some fallback (e.g., removing trailing 's').
        """
        topic_lower = topic.lower().strip()
        knowledge_list = []

        with self.driver.session() as session:
            # First attempt: exact search for the user-submitted topic
            query = """
                MATCH (m:Message)
                WHERE toLower(m.content) CONTAINS toLower($topic)
                RETURN m.content AS content
            """
            result = session.run(query, topic=topic_lower)
            knowledge_list = [record["content"] or "" for record in result]

            # If still no match, try a simpler variant:
            # e.g., if user typed "loops", also try "loop"
            if not knowledge_list:
                if topic_lower.endswith('s'):
                    alt_topic = topic_lower.rstrip('s')
                    alt_result = session.run(query, topic=alt_topic)
                    knowledge_list = [record["content"] or "" for record in alt_result]

        return knowledge_list

    def generate_three_difficulty_quiz(self, topic):
        """
        Generates exactly 3 multiple-choice questions (Easy, Medium, Hard) based on the given topic.
        If no matching data is found (even after fallback), instruct the model to rely on general knowledge.
        """
        knowledge_data = self.fetch_topic_knowledge(topic)

        if knowledge_data:
            # We found matching content in the DB, so use that knowledge directly:
            combined_knowledge = "\n".join(knowledge_data)
            prompt = (
                f"<s>[INST] Create exactly 3 multiple-choice questions about '{topic}':\n"
                f"1) Easy question\n"
                f"2) Medium question\n"
                f"3) Hard question\n\n"
                "For each question:\n"
                "- Provide 4 answer choices labeled A, B, C, D.\n"
                "- Clearly indicate which one is correct.\n\n"
                "Use ONLY the following knowledge:\n"
                f"{combined_knowledge}\n"
                "[/INST]"
            )
        else:
            # No direct/fallback match found, rely on general knowledge
            prompt = (
                f"<s>[INST] We have no direct data on '{topic}'. "
                "Using your own general knowledge, create exactly 3 multiple-choice questions "
                "about this topic, labeled (Easy, Medium, Hard). For each question:\n"
                "- Provide 4 answer choices labeled A, B, C, D.\n"
                "- Clearly indicate which one is correct.\n"
                "[/INST]"
            )

        # Tokenize and generate
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048
        ).to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(**inputs, max_length=2048)

        quiz_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return quiz_text

# =========================
# 4) Configure the Quiz Generator with your credentials
# =========================
# Replace these with your actual credentials and token
HF_TOKEN = "hf_lybRAUDJpsfqtJBMQQuVBTKfsRSQcnUcB" # DUMMY TOKEN CREATE YOUR OWN
URI = "neo4j+s://ac34d9e1.databases.neo4j.io"
USER = "neo4j"
PASSWORD = "tyd_z6syD5GXTsmZQKQVJHHbHghxEEeHlxF0FPXpe96SK4" # DUMMY PASSWORD CREATE YOUR OWN



quiz_generator = DifficultyQuizGenerator(URI, USER, PASSWORD, HF_TOKEN)

# =========================
# 5) Flask App Definition
# =========================
app = Flask(__name__)

@app.route("/")
def home():
    return "Welcome to the Quiz Generator API!"

@app.route("/quiz", methods=["GET"])
def get_quiz():
    """
    Usage example:
      GET /quiz?topic=loops
    will return a JSON response containing the quiz text.
    """
    topic = request.args.get("topic", None)
    if not topic:
        return jsonify({"error": "Please provide a 'topic' query parameter, e.g. /quiz?topic=loops"}), 400

    quiz_result = quiz_generator.generate_three_difficulty_quiz(topic)
    return jsonify({"quiz": quiz_result})

# =========================
# 6) Start the ngrok tunnel and run Flask
# =========================
public_url = ngrok.connect(5000)  # Expose port 5000 to the public internet
print(f"Public URL for API: {public_url}")

app.run(port=5000)